## Import libraries

In [1]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## Read object-feature matrix

In [3]:
df_expert_wisdm = pd.read_csv("../data/features/expert_wisdm.csv")
df_ar_wisdm = pd.read_csv("../data/features/ar_wisdm.csv")
df_ssa_wisdm = pd.read_csv("../data/features/ssa_wisdm.csv")

In [4]:
df_expert_uschad = pd.read_csv("../data/features/expert_uschad.csv")
df_ar_uschad = pd.read_csv("../data/features/ar_uschad.csv")
df_ssa_uschad = pd.read_csv("../data/features/ssa_uschad.csv")

# Classification

In [39]:
def get_internal_score(clf, X, y, max_iter=25):
    nb = np.unique(y).shape[0]
    scores = np.zeros(nb+1)
    for j in range(max_iter):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        scores[0] += accuracy_score(y_test, y_predict)
        for i in range(nb):
            scores[i+1] += accuracy_score(1*(np.array(y_test) == i), 
                                          1*(np.array(y_predict) == i))
            
    return scores / max_iter

In [40]:
def get_score(df, estimator, params_grid, test_size=0.3):
    X = df.iloc[:, 1:].values
    y = df['activity'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    clf = GridSearchCV(estimator, params_grid)
    clf.fit(X_train, list(y_train))
    clf_lr = clf.best_estimator_
    scores = get_internal_score(clf_lr, X, list(y))
    
    return scores

## Testing part 

In [27]:
parameters_lr = {'penalty': ['l1', 'l2'], 
                 'class_weight': ['balanced', None], 
                 'C': 10. ** np.arange(-1, 4, 1)}
parameters_svm = {'kernel': ['rbf'], 
                  'C': 10. ** np.arange(-1, 4, 1), 
                  'gamma': 10. ** np.arange(-3, 2, 1),
                  'class_weight': ['balanced', None]}

parameters_rf = {'n_estimators': [200], 
                 'class_weight': ['balanced', None], 
                 'max_depth': [None, 3, 5, 11]}

scores_wisdm = {}
scores_uschad = {}

**Expert** features:

In [ ]:
scores_wisdm['lr_expert'] = get_score(df_expert_wisdm, LogisticRegression(), parameters_lr)
scores_wisdm['svm_expert'] = get_score(df_expert_wisdm, SVC(), parameters_svm)
scores_wisdm['rf_expert'] = get_score(df_expert_wisdm, RFC(), parameters_rf)

In [ ]:
scores_uschad['lr_expert'] = get_score(df_expert_uschad, LogisticRegression(), parameters_lr)
scores_uschad['svm_expert'] = get_score(df_expert_uschad, SVC(), parameters_svm)
scores_uschad['rf_expert'] = get_score(df_expert_uschad, RFC(), parameters_rf)

From **autoregression model** features:

In [ ]:
n = 20

In [ ]:
scores_wisdm['lr_ar_' + str(n)] = get_score(df_ar_wisdm, LogisticRegression(), parameters_lr)
scores_wisdm['svm_ar_' + str(n)] = get_score(df_ar_wisdm, SVC(), parameters_svm)
scores_wisdm['rf_ar_' + str(n)] = get_score(df_ar_wisdm, RFC(), parameters_rf)

In [ ]:
scores_uschad['lr_ar_' + str(n)] = get_score(df_ar_uschad, LogisticRegression(), parameters_lr)
scores_uschad['svm_ar_' + str(n)] = get_score(df_ar_uschad, SVC(), parameters_svm)
scores_uschad['rf_ar_' + str(n)] = get_score(df_ar_uschad, RFC(), parameters_rf)

From **spectrum analysis** features:

In [ ]:
n = 20

In [ ]:
scores_wisdm['lr_ssa_' + str(n)] = get_score(df_ssa_wisdm, LogisticRegression(), parameters_lr)
scores_wisdm['svm_ssa_' + str(n)] = get_score(df_ssa_wisdm, SVC(), parameters_svm)
scores_wisdm['rf_ssa_' + str(n)] = get_score(df_ssa_wisdm, RFC(), parameters_rf)

In [ ]:
scores_uschad['lr_ssa_' + str(n)] = get_score(df_ssa_uschad, LogisticRegression(), parameters_lr)
scores_uschad['svm_ssa_' + str(n)] = get_score(df_ssa_uschad, SVC(), parameters_svm)
scores_uschad['rf_ssa_' + str(n)] = get_score(df_ssa_uschad, RFC(), parameters_rf)

## Results 

In [ ]:
results_wisdm = pd.DataFrame.from_dict(scores_wisdm, orient='index')
results_wisdm.columns = ['all'] + list(set(data_wisdm['activity']))
results_wisdm.to_csv("results_wisdm.csv", index=False)

results_uschad = pd.DataFrame.from_dict(scores_uschad, orient='index')
results_uschad.columns = ['all'] + list(set(data_uschad['activity']))
results_uschad.to_csv("results_uschad.csv", index=False)

In [ ]:
results_wisdm

In [ ]:
results_uschad